# 1. 피처 엔지니어링 2: 주소 수집하기

지역마다 호황기인 산업이 있고, 침체기인 산업이 존재한다. 따라서 우리는 주소를 수집하여 지역별 산업 현황을 살펴보고,

지역에 따른 산업군 분포를 파악하여 새로운 파생변수를 만들기 위해 주소 데이터를 수집한다.

기업의 정보를 검색할 수 있는 나이스 비즈인포에서 selenium을 통해 크롤링하여 주소 데이터를 수집하고자 한다.

### 병렬 처리를 적용한 크롤링 코드

- 주소를 수집해야할 데이터의 개수가 약 3만개로, 일반적인 크롤링을 시도하면 약 7일이 소요된다.
- 7일 동안 크롤링을 하기에는 무리가 있기 때문에 병렬 처리를 시도했다.
    - 7일 -> 11시간으로 크롤링 시간을 대폭 단축할 수 있었다.
- 현재 사용하는 CPU의 프로세서 개수는 12개 이므로, 데이터를 12개의 구간으로 나누어서 진행하였다.
- 산업코드 크롤링을 할 때와 동일하게 사업자등록번호, 기업명으로 검색해도 나오지 않는 지점인 기업들은 수작업으로 찾아서 데이터를 수집했다.

```python
# 필요한 라이브러리 불러오기
import json
import pandas as pd

import multiprocessing
from multiprocessing import Pool

from selenium import webdriver              
from selenium.webdriver import ActionChains  
from selenium.webdriver.common.by import By  
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException

def get_data(data):
    """
    나이스 비즈인포에서 사업자등록번호 or 기업명을 검색하여 주소를 수집하는 함수입니다.
    -------------------------------------------
    input = data(접속할 url, 검색할 데이터(list))
    -------------------------------------------
    """
    result_dict = [{'사업자등록번호':[], '기업명':[], '주소':[]}, {'사업자등록번호':[], '기업명':[]}] # 크롤링한 결과를 저장할 변수 선언
    # 예외처리를 위해 try ~ except문 사용
    try:
        # selenium으로 크롤링하기 위해 크롬 드라이버 불러오기
        dr = webdriver.Chrome()
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        dr.set_window_size(1000, 1000)                   
        dr.get(url=data[0]) 
        time.sleep(2)
        
        # 사업자등록번호, 기업명으로 검색해서 크롤링하기
        for cor_number, cor_name in zip(data[1], data[2]):
            # 01. 검색창에 사업자등록번호를 입력하고 검색버튼 누르기 (sleep=5)
            search_box_biz_info = dr.find_element(By.XPATH, "/html/body/div[2]/div/ul/li[1]/div/form/input")     # 비즈인포 검색창
            search_button_biz_info = dr.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[1]/div/form/button') # 비즈인포 검색 버튼
            time.sleep(2)

            act = ActionChains(dr)
            act.send_keys_to_element(search_box_biz_info, cor_name).click(search_button_biz_info).perform() # 사업자등록번호 입력, 버튼 클릭 수행
            time.sleep(5)

            # 예외처리를 위해 try ~ except문 사용
            try:
                # 02. 검색해서 나온 결과에서 주소 가져오기 (sleep=3)
                click_result = dr.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/table/tbody/tr[2]/td/div/ul/li[1]')
                time.sleep(2)

                result_dict[0]["사업자등록번호"].append(cor_number)
                result_dict[0]["기업명"].append(cor_name) 
                result_dict[0]["주소"].append(click_result.text)
                
            except NoSuchElementException as e:
                result_dict[1]["사업자등록번호"].append(cor_number)
                result_dict[1]["기업명"].append(cor_name)
                print(f'{cor_number} {cor_name}의 정보가 없습니다.')
            
            except UnexpectedAlertPresentException as e:
                result_dict[1]["사업자등록번호"].append(cor_number)
                result_dict[1]["기업명"].append(cor_name)
                print(f'{cor_number} {cor_name}의 정보가 없습니다.')

            # 중간저장
            filename = f'file_{data[1][0]}.json'
            with open(f'./custom_data/address_json_name_non2/{filename}','w') as f:
                json.dump(result_dict, f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(e)

    finally:
        dr.close()
        dr.quit()
        
if __name__ == '__main__':
    df = pd.read_csv('./crawling_data.csv', encoding='cp949')
    df.drop_duplicates(subset='사업자등록번호', inplace=True)  # 결산으로 인한 중복 데이터가 존재하므로 사업자등록번호를 기준으로 중복제거
    num_list = []
    name_list = []
    numbers = df['사업자등록번호'].values.tolist()
    names = df['기업명'].values.tolist()
    start = 0
    end = 2975
    # 데이터를 12개의 구간으로 나누기
    for i in range(0, 12):
        if i != 11:
            num_list.append(numbers[start:end])
            name_list.append(names[start:end])
            start = end
            end += 2975
        else:
            num_list.append(numbers[start:])
            name_list.append(names[start:])

    # 병렬 처리를 위한 CPU 나누기 및 데이터 만들기
    cpu_count = multiprocessing.cpu_count()
    url = 'https://www.nicebizinfo.com/cm/CM0100M001GE.nice'
    urls_list = [url] * cpu_count
    total = []
    for i, j, k in zip(urls_list, num_list, name_list):
        total.append([i, j, k])
    p = Pool(processes=cpu_count)
    p.map(get_data, total)
    
```

# 2. 수집한 주소 합치기

In [1]:
import json
import os

# 사업자등록번호로 검색한 결과물

path = "./custom_data/address_json/"
file_list = os.listdir(path)

data = []
for filename in file_list:
    with open(path+filename) as file_1:
	    data.append(json.load(file_1))

finish = data[0][0]
for i in range(1, len(data)):
    finish['사업자등록번호'].extend(data[i][0]['사업자등록번호'])
    finish['주소'].extend(data[i][0]['주소'])

retry = data[0][1]
for i in range(1, len(data)):
    retry['사업자등록번호'].extend(data[i][1]['사업자등록번호'])
    retry['기업명'].extend(data[i][0]['기업명'])

In [2]:
# 중간에 세션이 끊어져 누락된 데이터들 재검색(약 1000개)

path = "./custom_data/address_json_no/"
file_list = os.listdir(path)

data2 = []
for filename in file_list:
    with open(f'./custom_data/address_json_no/{filename}') as file_1:
	    data2.append(json.load(file_1))

for i in range(0, len(data2)):
    finish['사업자등록번호'].extend(data2[i][0]['사업자등록번호'])
    finish['주소'].extend(data2[i][0]['주소'])

for i in range(0, len(data2)):
    retry['사업자등록번호'].extend(data2[i][1]['사업자등록번호'])
    retry['기업명'].extend(data2[i][0]['기업명'])

In [3]:
# 사업자등록번호로 나오지 않아서 기업명으로 검색한 결과물

path = "./custom_data/address_json_name/"
file_list = os.listdir(path)

data3 = []
for filename in file_list:
    with open(f'./custom_data/address_json_name/{filename}') as file_1:
	    data3.append(json.load(file_1))
        
for i in range(0, len(data3)):
    finish['사업자등록번호'].extend(data3[i][0]['사업자등록번호'])
    finish['주소'].extend(data3[i][0]['주소'])

retry2 = data3[0][1]
for i in range(1, len(data3)):
    retry2['사업자등록번호'].extend(data3[i][1]['사업자등록번호'])
    retry2['기업명'].extend(data3[i][0]['기업명'])

In [4]:
# 중간에 세션이 끊어져 누락된 데이터들 재검색(약 100개)

path = "./custom_data/address_json_name_non/"
file_list = os.listdir(path)

data3 = []
for filename in file_list:
    with open(f'./custom_data/address_json_name_non/{filename}') as file_1:
	    data3.append(json.load(file_1))
        
for i in range(0, len(data3)):
    finish['사업자등록번호'].extend(data3[i][0]['사업자등록번호'])
    finish['주소'].extend(data3[i][0]['주소'])

retry2 = data3[0][1]
for i in range(1, len(data3)):
    retry2['사업자등록번호'].extend(data3[i][1]['사업자등록번호'])
    retry2['기업명'].extend(data3[i][0]['기업명'])

In [5]:
len(finish['사업자등록번호']), len(finish['기업명'])

(29197, 1417)

In [6]:

len(retry2['사업자등록번호']), len(retry2['기업명'])

(239, 118)

## 2-1. 수작업으로 수집한 주소 데이터 형식 바꾸기

In [7]:
# 딕셔너리 형태인데, key가 기업명이므로, 사업자등록번호로 key를 바꿔준다.

# 바꿔질 데이터
with open(f'./custom_data/search_address.json', encoding='utf-8') as file_1:
	search_address = json.load(file_1)

# key값을 바꾸기 위해 사용될 데이터
with open(f'./custom_data/search_address.json', encoding='utf-8') as file_1:
	revise_address = json.load(file_1)

In [10]:
import pandas as pd

df = pd.read_csv('./custom_data/active_closed_all.csv', encoding='cp949')
df.drop_duplicates(subset='사업자등록번호', inplace=True)

In [11]:
df['사업자등록번호'].nunique()

35721

In [12]:
# 사업자번호추출하기
for search_result in search_address:
    if len(df.loc[df['기업명']==search_result, '사업자등록번호'].values) == 1:
        cor_numbers = df.loc[df['기업명']==search_result, '사업자등록번호'].values[0]
        revise_address[search_result] = cor_numbers

In [13]:
# 추출한 사업자등록번호로 key 값 바꾸기
fianl_address = dict((revise_address[key], value) for (key, value) in search_address.items())

# 3. 원본 데이터에 주소 데이터 합치기

In [14]:
# 중복이 제거되지 않은 데이터로 다시 불러오기
df = pd.read_csv('./custom_data/active_closed_all.csv', encoding='cp949')

In [15]:
# 크롤링으로 수집한 데이터들 합치기
for cor_number, cor_address in zip(finish['사업자등록번호'], finish['주소']):
    df.loc[df['사업자등록번호']==cor_number, "주소"] = cor_address

In [16]:
# 수작업으로 수집한 데이터들 합치기
for cor_number in fianl_address:
    df.loc[df['사업자등록번호']==cor_number, "주소"] = fianl_address[cor_number]

In [17]:
import numpy as np
df['주소'] = df['주소'].replace('', np.nan)

In [18]:
df = df.dropna(subset=['주소'])

# 4. 내보내기

In [17]:
#df.to_csv('./custom_data/feature_engineering_address.csv', encoding='utf-8', index=False)

In [19]:
df['주소'].isna().sum()

0

In [20]:
df.loc[df['주소'] == "서울특별시\xa0중구\xa0을지로1가\xa0100-1\xa0", "주소"] = "서울특별시 중구 을지로1가 100-1"

In [21]:
transform_address = {"서울":"서울특별시", 
                     "서울시":"서울특별시",
                     "경기":"경기도",
                     "부산":"부산광역시",
                     "경남":"경상남도",
                     "인천":"인천광역시",
                     "경북":"경상북도",
                     "충남":"충청남도",
                     "대구":"대구광역시",
                     "충북":"충청북도",
                     "광주":"광주광역시",
                     "전남":"전라남도",
                     "대전":"대전광역시",
                     "울산":"울산광역시",
                     "전북":"전라북도",
                     "강원":"강원도",
                     "제주":"제주특별자치도",
                     "세종":"세종특별자치시"}

In [22]:
address_list = []
def address_aply(value):
    if value.split(' ')[0] == "제주시":
        return "제주특별자치도 " + value
    elif value.split(' ')[0] == "대전공장":
        return value[5:]
    elif value.split(' ')[0] == "아산군":
        return "충청남도 " + value
    elif value.split(' ')[0] == "\t충청남도":
        return value.replace('\t', '')
    return value
df['주소'] = df['주소'].apply(address_aply)

In [23]:
address_list = []
def address_aply(value):
    if value.split(' ')[0] in transform_address.keys():
        return transform_address[value.split(' ')[0]]
    return value.split(' ')[0]
df['시도'] = df['주소'].apply(address_aply)

In [24]:
df['시도'].value_counts()

서울특별시      8343
경기도        7803
부산광역시      1994
경상남도       1739
인천광역시      1525
경상북도       1186
충청남도       1167
대구광역시      1014
충청북도        862
광주광역시       685
전라남도        603
대전광역시       582
울산광역시       523
전라북도        513
강원도         423
제주특별자치도     326
세종특별자치시     141
Name: 시도, dtype: int64

In [25]:
address_list = []
def address_aply(value):
    if value.split(' ')[0] == '세종':
        return np.nan
    elif value.split(' ')[1] == "창원":
        return '창원시'
    elif value.split(' ')[1] == '광양':
        return '광양시'
    return value.split(' ')[1]
df['시군구'] = df['주소'].apply(address_aply)

In [26]:
df['시군구'].unique()

array(['구로구', '광진구', '연수구', '경산시', '중구', '서초구', '종로구', '금천구', '양산시',
       '달서구', '서구', '영등포구', '동구', '김제시', '창원시', '강북구', '안성시', '성동구',
       '김해시', '강남구', '제주시', '안산시', '함안군', '청주시', '예산군', '남구', '강동구',
       '강서구', '서귀포시', '울주군', '사상구', '고양시', '광주시', '충주시', '남동구', '북구',
       '음성군', '오산시', '영천시', '의왕시', '서대문구', '안양시', '보령시', '원주시', '파주시',
       '천안시', '아산시', '동래구', '송파구', '당진시', '시흥시', '목포시', '포항시', '유성구',
       '해운대구', '달성군', '마포구', '창녕군', '과천시', '화성시', '양주시', '대덕구', '김포시',
       '평택시', '동해시', '구미시', '춘천시', '성남시', '사하구', '광명시', '군산시', '여수시',
       '광양시', '경주시', '미추홀구', '포천시', '남양주시', '수성구', '광산구', '영도구', '김천시',
       '고성군', '수영구', '서산시', '군포시', '용인시', '칠곡군', '곡성군', '익산시', '용산구',
       '금정구', '부평구', '수원시', '예천군', '하남시', '양천구', '부천시', '서천군', '통영시',
       '횡성군', '이천시', '여주시', '정선군', '순천시', '화순군', '성북구', '논산시', '계양구',
       '중랑구', '강화군', '부산진구', '연제구', '강릉시', '동대문구', '진천군', '가평군', '동두천시',
       '기장군', '고령군', nan, '공주시', '진주시', '성주군', '군위군', '거제시', '거창군', '영주시',
       '밀양시

In [27]:
df[['시도', '시군구']]

,시도,시군구
0,서울특별시,구로구
1,서울특별시,광진구
2,인천광역시,연수구
3,경상북도,경산시
4,서울특별시,중구
...,...,...
35716,서울특별시,성동구
35717,경상북도,문경시
35718,경기도,고양시
35719,경기도,부천시


In [29]:
df = df.reset_index(drop=True)

In [ ]:
df.to_csv('./custom_data/address.csv')

In [27]:
# 추가한 파생변수들이 있는 데이터프레임 내보내기
code_df = pd.read_csv('./custom_data/feature_engineering_idurstry_code.csv', encoding='cp949')

In [28]:
code_df

,사업자등록번호,기업접두명,기업명,기업접미명,기업영문명,업종,기업규모,공기업구분 1.공기업 2.일반기업,개인법인구분 1.법인 2.개인,본점지점구분 1.본점 2.지점,...,국가명,홈페이지URL,대표자명,직원수,종료일자,시작일자,휴폐업구분,상태발생일자,업종중분류,업종대분류
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",제조,중소기업,일반기업,법인,본점,...,NaN,www.hanilgas.com,홍순철외 1명,43.0,NaN,NaN,NaN,NaN,소매업; 자동차 제외,도매 및 소매업
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",제조,중소기업,일반기업,법인,본점,...,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35.0,NaN,NaN,NaN,NaN,"가죽, 가방 및 신발 제조업",제조업
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",제조,중소기업,일반기업,법인,본점,...,NaN,transport@naver.com,김명화,149.0,NaN,NaN,NaN,NaN,육상 운송 및 파이프라인 운송업,운수 및 창고업
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",제조,중소기업,일반기업,법인,본점,...,NaN,www.koreajw.com,김명구,75.0,NaN,NaN,NaN,NaN,자동차 및 트레일러 제조업,제조업
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",제조,중소기업,일반기업,법인,본점,...,NaN,NaN,김홍득,3.0,NaN,NaN,NaN,NaN,도매 및 상품 중개업,도매 및 소매업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35716,4808501629,NaN,게방식당팩토리,NaN,gebangsikdangfactory,제조,중소기업,일반기업,법인,지점,...,NaN,https://gebangsikdang.modoo.at/,방건혁,NaN,99991231.0,20211231.0,폐업,20211231.0,식료품 제조업,제조업
35717,8018501578,(주),우진푸드중부지점,NaN,woojinfood,제조,중소기업,일반기업,법인,지점,...,NaN,우진푸드.com,신남정,22.0,99991231.0,20211126.0,폐업,20211126.0,식료품 제조업,제조업
35718,4118523419,(주),윤월드푸드코리아다크써클브로스,NaN,"Youn World Food Korea Co., Ltd.",제조,중소기업,일반기업,법인,지점,...,NaN,NaN,윤인수,0.0,99991231.0,20220630.0,폐업,20220630.0,음식점 및 주점업,숙박 및 음식점업
35719,1738102533,(주),기린테크,NaN,"Girin Tech Co., Ltd.",제조,중소기업,일반기업,법인,본점,...,NaN,NaN,임계현,NaN,99991231.0,20220731.0,폐업,20220731.0,출판업,정보통신업


In [30]:
a = pd.merge(df, code_df[['사업자등록번호', '산업코드1', '업종중분류', '업종대분류', '휴폐업구분']],  how='inner', on='사업자등록번호')

In [31]:
a.to_csv('./finance_address_indurstry.csv', index=False)

In [32]:
a.shape

(77350, 219)

In [33]:
pd.read_csv('./finance_address_indurstry.csv')

,사업자등록번호,결산년월,총자본증가율,영업이익증가율,당기순이익증가율,자기자본증가율,매출액증가율,매출총이익률,매출액영업이익률,매출액경상이익률,...,매출채권대매입채무비율(top10_비율),매출채권대매입채무비율(all_차이),매출채권대매입채무비율(all_비율),주소,시도,시군구,산업코드1,업종중분류,업종대분류,휴폐업구분
0,1018100340,20191231,0.037308,1.013936,0.089367,0.114629,-0.013259,1.000000,0.128879,0.526796,...,0.000000,-0.758599,0.000000,서울 종로구 삼일대로 428,서울특별시,종로구,68112.0,부동산업,부동산업,NaN
1,1018100340,20201231,0.117880,-0.919169,0.246335,0.181063,-0.092985,1.000000,0.011485,0.704525,...,0.000000,-1.389440,0.000000,서울 종로구 삼일대로 428,서울특별시,종로구,68112.0,부동산업,부동산업,NaN
2,1018100340,20211231,-0.056428,4.678983,0.122320,-0.013509,0.037967,1.000000,0.062839,0.768272,...,0.000000,-7.878755,0.000000,서울 종로구 삼일대로 428,서울특별시,종로구,68112.0,부동산업,부동산업,NaN
3,1018100772,20191231,0.028081,0.541345,-0.427849,0.001721,0.020793,0.549552,0.208576,0.010025,...,0.000000,-0.438342,0.000000,서울 종로구 청계천로 137,서울특별시,종로구,55101.0,숙박업,숙박 및 음식점업,NaN
4,1018100772,20201231,0.008062,-3.774965,-63.241980,-0.106922,-0.424458,-0.045240,-1.005644,-1.084150,...,0.000000,-0.393467,0.000000,서울 종로구 청계천로 137,서울특별시,종로구,55101.0,숙박업,숙박 및 음식점업,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77345,8998800427,20201231,0.149454,0.278016,-0.076174,0.192417,0.188646,0.150174,0.067453,0.032470,...,1.351139,0.335617,1.351139,충남 천안시 서북구 입장면 성진로 1192-6,충청남도,천안시,26299.0,"전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",제조업,NaN
77346,8998800427,20211231,0.249737,-2.215573,-5.213883,-0.610733,0.334008,0.071243,-0.061465,-0.089610,...,1.052860,0.057015,1.052860,충남 천안시 서북구 입장면 성진로 1192-6,충청남도,천안시,26299.0,"전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",제조업,NaN
77347,8998800785,20191231,0.000000,-0.019334,-0.012369,-0.074572,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.758599,0.000000,서울 중구 남대문로 84,서울특별시,중구,68000.0,부동산업,부동산업,NaN
77348,8998800785,20201231,0.000000,-0.393148,-0.392873,-0.048923,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.389440,0.000000,서울 중구 남대문로 84,서울특별시,중구,68000.0,부동산업,부동산업,NaN
